In [31]:
import sys
import numpy as np
import pickle
from mne_bids import BIDSPath, read_raw_bids
from bids import BIDSLayout
from util.io.coherence import *
from util.io.iter_BIDSPaths import *
from mne_connectivity import spectral_connectivity_time, check_indices
from scipy.signal import coherence

In [3]:
SUB = '31'
RUN = '1'
TASK = 'pitch'
FPATH = '/project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz'

BIDS_ROOT = '../data/bids'
FIGS_ROOT = '../figs'

DERIV_ROOT = '../data/bids/derivatives'
METHOD = 'coh'
FS = 5000
RAW_TMIN = -0.2
RAW_TMAX = 0.5
TMIN = 0
TMAX = 0.25
N_CHANS = 62
CONDS = ['50', '100', '150', '200', '250']
FREQS = [50, 100, 150, 200, 250]

In [4]:
# Load epoched data
epochs = mne.read_epochs(FPATH, preload = True)
events = epochs.events
n_epochs = len(events)

Reading /project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Reading /project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif-1.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
4801 matching events found
No baseline correction applied
0 projection items activated


In [5]:
# Use a different sub for generating stim channels if sub has bad Aux channel
STIM_SUB, STIM_RUN = get_stim_sub(SUB, RUN)

In [21]:
# Create epochs from raw data to create simulated stim channels
raw_epochs = get_raw_epochs(BIDS_ROOT, STIM_SUB, TASK, STIM_RUN)
stim_epochs_array = create_stim_epochs_array(raw_epochs, n_epochs, CONDS)
simulated_epochs = create_stim_epochs_object(stim_epochs_array, events, CONDS, FS, RAW_TMIN)

Used Annotations descriptions: ['100', '150', '200', '250', '50']
Not setting metadata


/project2/hcn1/pitch_tracking/scripts/util/io/coherence.py:42: RuntimeWarning: The unit for channel(s) Aux1 has changed from NA to V.
  raw = read_raw_bids(bids_path, verbose = False)
/project2/hcn1/pitch_tracking/scripts/util/io/coherence.py:42: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Aux1']
  raw = read_raw_bids(bids_path, verbose = False)
/project2/hcn1/pitch_tracking/scripts/util/io/coherence.py:42: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['Aux1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


6000 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1180 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1234 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1216 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1219 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1151 events and 3501 original time points ...
0 bad epochs dropped
Not setting metadata
4801 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


In [23]:
# Crop data so both epoch objects have same windowing
simulated_epochs = simulated_epochs.crop(tmin = TMIN, tmax = TMAX)
epochs = epochs.crop(tmin = TMIN)

In [28]:
# Keep only one version of stim signals
stim_array = simulated_epochs.get_data()[0, :, :]

In [86]:
# epochs is (4801, 62, 1251) which is (epochs, channels, time points)
# simulated epochs is (5, 1251) which is (freqs, time points)

epoch_array = epochs.get_data()
n_epochs = np.shape(epoch_array)[0]
n_chans = np.shape(epoch_array)[1]
n_freqs = 5

df = pd.DataFrame()

for epoch in range(n_epochs):
    for channel in range(n_chans):
        current_epoch = epoch_array[epoch, channel, :]
        for freq in range(n_freqs):
            current_freq = stim_array[freq, :]
            
            # compute coherence
            f, Cxy = coherence(current_epoch, current_freq, FS)
            
            # get coherences for condition frequencies only
            epoch_df = extract_epoch_coherence(epoch, FREQS, f, Cxy) 
            df = pd.concat([df, epoch_df])
            break
        break
    break

   epoch       50       100       150       200      250
0      0  0.17383  0.231918  0.143334  0.070176  0.13471
   epoch       50       100       150       200      250
0      0  0.17383  0.231918  0.143334  0.070176  0.13471


In [84]:
# What shape does it need to be for decoding?? Keep this in mind!

In [83]:
epoch_df

,epoch,50,100,150,200,250
0,0,0.17383,0.231918,0.143334,0.070176,0.13471
